# 階層的クラスタリング

この演習では下記のデータを用いて，階層的クラスタリングを体験します．
* とある購買データ
* 2018年度プロ野球打者成績データ

演習に必要な以下のライブラリを読み込んでおきます．

In [ ]:
# 表形式のデータを操作するためのライブラリ
import pandas as pd

# 行列計算をおこなうためのライブラリ
import numpy as np

# 距離行列の計算のためのライブラリ
import scipy.spatial.distance as distance

# 階層的クラスタリング用のライブラリ
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree

# グラフ描画ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns;
sns.set(style='ticks')
%matplotlib inline

また，この演習では日本語を含むグラフを生成するため，問題が生じないよう以下のおまじないコードを実行しておいてください．

In [ ]:
!pip install japanize-matplotlib
import japanize_matplotlib 

---
## 例題1: とある購買データ

あるeコマースサイトにおける購買データを用いて階層的クラスタリングを実行してみましょう．
``data``フォルダにある``e-commerce.txt``ファイルは，
* 書籍
* 衣服
* 化粧品
* 食料品
* 飲料

を1年間で何回購入したかについて，100名の顧客のデータが記録されています．
以下のコードを実行して，データを読み込みましょう．

In [ ]:
url = "http://drive.hontolab.org/ymc-lecture2020/data/e_commerce.txt"
e_commerce_df = pd.read_table(url, header=0, sep=" ")

### ベクトルデータ間の距離の計算

Pythonで階層的クラスタリングを実行するには`scipy`ライブラリの``linkage``関数を用いますが，``linkage``関数は入力（第1引数）として **ベクトル間の距離情報（を格納した行列データ）** を求めます．
ここでベクトルとは，n個の特徴量からなるデータを意味します．今回の``e_commerce.txt``ファイルでは，1行1行が1名の顧客情報を表すベクトルデータとなります．

正攻法でやるならば，100行ある``e_commerce_df``の顧客データのすべての組み合わせを比較して，各顧客ベクトルの距離を計算する必要があります．
しかし，`scipy`ライブラリはそれを簡単に実行してくれる便利な関数``distance.pdist``を提供しています．

以下のコードを実行してベクトル間の距離を計算してみましょう．

In [ ]:
dist_matrix = distance.pdist(e_commerce_df, metric='euclidean')

# 表示したい場合は以下のコードを#記号を削除して実行
#pd.DataFrame(distance.squareform(dist_matrix))

100行あるベクトルデータのすべての組み合わせについて距離を計算した結果（距離行列）が``dist_matrix``として得られました．
この距離行列を使って階層的クラスタリングを実行してみましょう．

### 階層的クラスタリングの実行

講義でも述べたように，階層的クラスタリングではクラスタの作り方の基準として
* 最長距離法（complete linkage method）
* 最短距離法（single linkage method）
* セントロイド法（centroid method）
* ウォード法（Ward's method）

などがあります．今回はウォード法を使ってみましょう．以下のコードを実行してみてください．

In [ ]:
e_commerce_result = linkage(dist_matrix, method="ward")

階層的クラスタリングの結果が変数``e_commerce_result``に格納されました．

階層的クラスタリングにおいて，各要素（ベクトル）が徐々に併合されていく結果を示したものを**デンドログラム**と呼びます．デンドログラムを表示させてみましょう．以下のコードを実行してみてください．


In [ ]:
# 図の大きさの設定
plt.figure(figsize=(10, 14))

# 図の表示
fig = dendrogram(e_commerce_result,
                 leaf_font_size=8, orientation='left')
plt.show()

デンドログラムが表示されると，階層的クラスタリングを実行したという気分になりますね！

今回は顧客の一人一人について私たちは特に知識がないため，この図を眺めてもよく分かりません．そこで，適当な深さでデンドログラムの枝を切って，各要素をクラスタに分けてみましょう．つまりK-meansクラスタリングと同じようなことをします．その上で，
各クラスタに入っている要素がどのような性質を持っているかを分析していましょう．

デンドログラムの枝を切り，指定したクラスタ数に分割するには``cutree``関数を用います．``k``に数値を指定すると，デンドログラムをその数のクラスタに分割することができます．


In [ ]:
# 4つのクラスタに分割
e_commerce_cluster = cut_tree(e_commerce_result, n_clusters=4)

# 最初の10件のみ表示
e_commerce_cluster[:10, 0]

変数``e_commerce_cluster``に100名の顧客のクラスタ割り当て番号が格納されました．

### クラスタの分析

さて，クラスタリングの結果は変数``e_commerce_cluster``に得られましたが，各クラスタがどのような購買傾向を持っているかを分析するには，``e_commerce_df``データと照らし合わせる必要があります．
そこで，以下のコードを実行して，``e_commerce_cluster``と``e_commerce_df``を結合してみましょう．

In [ ]:
# 結合
new_e_commerce_df = e_commerce_df.assign(
    cluster_id = e_commerce_cluster[:, 0]
)

# 先頭の数件のみ表示
new_e_commerce_df.head()

各顧客データがどのクラスタに分類するかのデータが得られました．このデータを用いて，クラスタ毎に書籍の平均購買数，衣類の平均購買数などを調べてみましょう．

今回の分析のように，クラスタ毎に何らかの情報をまとめて計算することを**集約演算**と呼びます．
やや複雑に見えますが，以下のコードを実行してみてください．

In [ ]:
new_e_commerce_df.groupby(
    'cluster_id' # cluster_idでデータをまとめる
).agg(
    'mean' # まとまり毎に平均値を算出
)

クラスタ毎に書籍，衣類，化粧品，食料品，飲料の平均購買数が求まりました．クラスタ毎に少しずつ特徴が異なることが確認できます．

---
## 例題2: 2018年度プロ野球打者成績データ

例題1のデータは見ず知らずの顧客のデータであったため，クラスタリングをしても予想される結果がイメージしにくかったと思います．そこで次は実データを分析してみましょう．

下記コマンドを実行して，``baseball_stats_2018.tsv``ファイルを読み込みます．
このデータは2018年度のプロ野球において，規定打席に達した打者の成績（例：「打率」「出塁率」「長打率」）を記録したものです．

In [ ]:
# データの読み込み
url = "http://drive.hontolab.org/ymc-lecture2020/data/baseball_stats_2018.tsv"
stats_df = pd.read_table(url, header=0, sep="\t", index_col='選手名')

# 最初の数件を表示
stats_df.head()

このデータに対して，階層的クラスタリングを実行して傾向が似通った選手をグルーピングしてみましょう．

データをご覧になると分かるように，``stats_df``には「打率」「安打数」「ホームラン数」といった成績情報以外に「チーム名」（とデータのラベルとして「選手名」）が含まれています．
今回の分析では野手成績に関係する「打率」〜「長打率」だけに焦点をしぼって分析をします．

``stats_df``では「打率」〜「長打率」は1〜17列目（0列スタート）に格納されています．
``stats_df``から1〜17列目だけを抜き出すには，下記のようなコードを書きます．

In [ ]:
stats_df.iloc[:, 1:17]

この方法を使って，選手データをクラスタリングしてみましょう．

例題1と同様，今回の分析では階層的クラスタリングのタイプとして**ウォード法**を用いることにします．
下記のコードを実行すると，階層的クラスタリングを行い，その結果をデンドログラムとして表示します．

In [ ]:
# 距離行列の計算
stats_dist_matrix = distance.pdist(stats_df.iloc[:, 1:17], metric='euclidean')

# 階層的クラスタリングの実行
stats_result = linkage(stats_dist_matrix, method="ward")

# デンドログラムの表示
plt.figure(figsize=(10, 10))
fig = dendrogram(stats_result, labels=stats_df.index, 
                leaf_font_size=9, orientation='left')
plt.show()

デンドログラムが表示されました．プロ野球に関心のある方からご覧になって，このクラスタリング結果は妥当なものでしょうか？

例題1と同様に，各クラスタの平均的なスコアを計算してみましょう．
図を見ると，今回の結果から，データはざっくり見ると6クラスタに分かれそうです．
とりあえず6クラスタに分けたときに，各クラスタの平均スコアがどうなっているかを分析してみましょう．

In [ ]:
# 6つのクラスタに分割
stats_cluster = cut_tree(stats_result, n_clusters=6)

# クラスタ情報と選手データを結合
new_stats_df = stats_df.assign(
    cluster_id = stats_cluster[:, 0]
)

# 各クラスタの平均スコアを計算
new_stats_df.groupby(
    'cluster_id' # cluster_idでデータをまとめる
).agg(
    'mean' # まとまり毎に平均値を算出
)

各クラスタの特徴は以下のような感じでしょうか：
* 第1クラスタ: 打率も高く，ホームランも盗塁もできるユーティリティプレーヤー
* 第2クラスタ: ホームランヒッター
* 第3クラスタ: 打率もホームランもそこそこ高い6番バッタータイプ
* 第4クラスタ: これといって特徴はない
* 第5クラスタ: 出塁率が高く盗塁も多く足が速い，リードオフ・マンタイプ
* 第6クラスタ: 打率は高くないが，堅実に犠打でランナーを進める，2番バッタータイプ